In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd


In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}


#extaraction le nombre des pages pour preparer l'extraction global 
url = 'https://www.avito.ma/fr/maroc/voitures-à_vendre'
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text ,'html')
#extraction tout les balises <a> 
balises_a = soup.find_all('a',href=True)

#extraction le dernier nombre de pagination
nombres_entiers = []

for balise_a in balises_a:
    texte = balise_a.text.strip()
    if re.match(r'^-?\d+$', texte):
        nombres_entiers.append(int(texte))

lastpage = int(nombres_entiers[-1])
print(lastpage)

#extraction tout les lignes pour tout les annonces des voitures jusqu'à la deriniere page
all_liens =[]
for i in range(1):
    url = 'https://www.avito.ma/fr/maroc/voitures-à_vendre?o='+str(1)
    balises_a = soup.find_all('a',href=True,class_='sc-1jge648-0 eTbzNs')
    all_liens.extend(balises_a)
#tout les href qui rederige vers l'annonce 
hrefs = []
for link in all_liens:
    href = link.get('href')
    hrefs.append(href)

#tout les voitures ont extracté
liste_voitures = []
#pour chaque annonces seul
for href in hrefs :
    url = href
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text ,'html')

    prixc = soup.find('p', attrs={'class': 'sc-1x0vz2r-0 lnEFFR sc-1g3sn3w-13 czygWQ'})
    if(prixc is None) :
        prixc = "NAN"
    else :
        prix = prixc.text
        prix_val= prix.replace(' ', '').replace(' ', '').replace('DH', '')
#les 3 premiers infos
    first_infos = soup.find_all('span', attrs={'class': 'sc-1x0vz2r-0 kQHNss'})
    for info in first_infos :
        if info.text.lower() == "automatique" or info.text.lower() == "manuel":
            transmission = info.text
        elif info.text.lower() == "diesel" or info.text.lower() == "essence":
            typeCarburant = info.text
        elif "cv" in info.text.lower():
            nbr_cv = info.text

#tout les noms "caracteristiques" et leur valeurs equivalents
    span_carac = soup.find_all('span',attrs={'class': 'sc-1x0vz2r-0 jZyObG'})
    span_carac_val = soup.find_all('span',attrs={'class': 'sc-1x0vz2r-0 gSLYtF'})
    
#compt pour chaque carac et sa valeur precis
    i = 0 
    voiture = {}
    for sc in span_carac :
        if sc.text == "Type":
            type = span_carac_val[i].text
            voiture [sc.text] = type
        if sc.text == "État":
            etat = span_carac_val[i].text
            voiture [sc.text] = etat
        if sc.text == "Marque":
            marque = span_carac_val[i].text
            voiture [sc.text] = marque
        if sc.text == "Modèle":
            modele = span_carac_val[i].text
            voiture [sc.text] = modele
        if sc.text == "Année-Modèle":
            anne_model = span_carac_val[i].text
            voiture [sc.text] = anne_model
        if sc.text == "Première main":
            premier_main = span_carac_val[i].text
            voiture [sc.text] = premier_main
        if sc.text == "Nombre de portes":
            nbr_portes = span_carac_val[i].text
            voiture [sc.text] = nbr_portes
        if sc.text == "Kilométrage":
            Kilométrage = span_carac_val[i].text
            voiture [sc.text] = Kilométrage
        i += 1
        voiture ["prix en (DHS)"] = prix_val
    liste_voitures.append(voiture)
    
#stocker les donnees extracte dans dataframe 
df = pd.DataFrame(liste_voitures)
display(df)
#save as csv
df.to_csv('avito_data_voitures_struc121t.csv', index=False , sep=';')  


# nbr_cv = soup.find('span', attrs={'class': 'sc-1x0vz2r-0 kQHNss'})
# print(nbr_cv)




2642


NameError: name 'prix_val' is not defined